In [5]:
import logging

logging.basicConfig(level=logging.INFO)
import json
from itertools import combinations

import eo_tools
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import shapely
from eo_tools.S1.process import process_insar
from eodag import EODataAccessGateway, EOProduct
from shapely.geometry import shape

dag = EODataAccessGateway()
# make sure cop_dataspace will be used
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

INFO:eodag.config:Loading user configuration from: /home/figi/.config/eodag/eodag.yml
INFO:eodag.core:usgs: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:aws_eos: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:meteoblue: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:hydroweb_next: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:wekeo: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:creodias_s3: provider needing auth for search has been pruned because no crendentials could be found
INFO:eodag.core:Locations configuration loaded from /home/figi/.config/eodag/locations.yml


In [3]:
# Run search

bbox = {
    "lonmin": 0.08905898091569497,
    "latmin": 52.15527412683906,
    "lonmax": 0.9565339502005088,
    "latmax": 52.69722175598818,
}
shp = shapely.box(bbox["lonmin"], bbox["latmin"], bbox["lonmax"], bbox["latmax"])

search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-04-01",
    "end": "2023-06-30",
    "geom": bbox,
}

results, _ = dag.search(**search_criteria)

# Find overlaps
data = []
for item in results:
    id = item.properties["id"]
    geom = shape(item.geometry)
    data.append({"id": id, "geometry": geom})

gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")  # Assuming WGS84

# 98% overlap
threshold = 0.98

overlaps = []
for (idx1, row1), (idx2, row2) in combinations(gdf.iterrows(), 2):
    intersection = row1["geometry"].intersection(row2["geometry"])
    if not intersection.is_empty:
        # Calculate overlap ratio as the area of intersection divided by the area of the smaller polygon
        overlap_ratio = intersection.area / min(
            row1["geometry"].area, row2["geometry"].area
        )
        if overlap_ratio >= threshold:
            overlaps.append((row1["id"], row2["id"], overlap_ratio))

overlap_ids = [entry[:-1] for entry in overlaps]
overlap_ids

INFO:eodag.core:Searching product type 'S1_SAR_SLC' on provider: cop_dataspace
INFO:eodag.search.qssearch:Sending search request: http://catalogue.dataspace.copernicus.eu/resto/api/collections/Sentinel1/search.json?startDate=2023-04-01&completionDate=2023-06-30&geometry=POLYGON ((0.0891 52.1553, 0.0891 52.6972, 0.9565 52.6972, 0.9565 52.1553, 0.0891 52.1553))&productType=SLC&maxRecords=20&page=1&exactCount=1
INFO:eodag.core:Found 39 result(s) on provider 'cop_dataspace'


[('S1A_IW_SLC__1SDV_20230401T174201_20230401T174228_047906_05C1B2_D88C',
  'S1A_IW_SLC__1SDV_20230624T174205_20230624T174232_049131_05E86A_1858'),
 ('S1A_IW_SLC__1SDV_20230401T174136_20230401T174203_047906_05C1B2_746D',
  'S1A_IW_SLC__1SDV_20230519T174138_20230519T174205_048606_05D890_932B'),
 ('S1A_IW_SLC__1SDV_20230401T174136_20230401T174203_047906_05C1B2_746D',
  'S1A_IW_SLC__1SDV_20230624T174140_20230624T174207_049131_05E86A_2C04'),
 ('S1A_IW_SLC__1SDV_20230401T174136_20230401T174203_047906_05C1B2_746D',
  'S1A_IW_SLC__1SDV_20230531T174139_20230531T174206_048781_05DDD1_5B0E'),
 ('S1A_IW_SLC__1SDV_20230401T174136_20230401T174203_047906_05C1B2_746D',
  'S1A_IW_SLC__1SDV_20230507T174137_20230507T174204_048431_05D35D_9687'),
 ('S1A_IW_SLC__1SDV_20230403T061502_20230403T061529_047928_05C270_1CD3',
  'S1A_IW_SLC__1SDV_20230521T061504_20230521T061532_048628_05D942_19C5'),
 ('S1A_IW_SLC__1SDV_20230403T061502_20230403T061529_047928_05C270_1CD3',
  'S1A_IW_SLC__1SDV_20230602T061505_20230602T

In [8]:
# download a single product
dag.download(
    EOProduct.from_geojson(results[0].as_dict()),
    outputs_prefix="data",
    extract=False,
)

0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(5e9e0ddd-4aa5-40d8-acf1-da497526abcf)/$value


KeyboardInterrupt: 

In [9]:
# Set download dirs
data_dir = "data"

ids = [
    "S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F",
    "S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033",
]

primary_dir = f"{data_dir}/{ids[0]}.zip"
secondary_dir = f"{data_dir}/{ids[1]}.zip"
outputs_prefix = f"{data_dir}/res/test-full-processor"

In [10]:
# Download overlapping products
to_dl = [it for it in results if it.properties["id"] in ids]
print(f"{len(to_dl)} products to download")
dag.download_all(to_dl, outputs_prefix=data_dir, extract=False)

INFO:eodag.core:Downloading 2 products


2 products to download


Downloaded products:   0%|          | 0/2 [00:00<?, ?product/s]

0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(560dc4f0-4661-4ff5-a7e2-3c2280c87c81)/$value
INFO:eodag.download.base:Product already downloaded: /home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.zip
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(560dc4f0-4661-4ff5-a7e2-3c2280c87c81)/$value


0.00B [00:00, ?B/s]

INFO:eodag.download.base:Download url: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(a283b833-9652-4746-a7ff-5a33ef70b755)/$value
INFO:eodag.download.base:Product already downloaded: /home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033.zip
INFO:eodag.download.base:Extraction not activated. The product is available as is.
INFO:eodag.product:Remote location of the product is still available through its 'remote_location' property: https://catalogue.dataspace.copernicus.eu/odata/v1/Products(a283b833-9652-4746-a7ff-5a33ef70b755)/$value


['/home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.zip',
 '/home/figi/software/work/eodh/user-workflows/test-1/data/S1A_IW_SLC__1SDV_20230916T174209_20230916T174236_050356_061016_8033.zip']

In [6]:
eo_tools.util.explore_products(results, shp)

In [12]:
out_dir = process_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vv",
    subswaths=["IW1", "IW2", "IW3"],
    write_coherence=True,
    write_interferogram=True,
    write_primary_amplitude=True,
    write_secondary_amplitude=False,
    apply_fast_esd=True,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    boxcar_coherence=[3, 3],
    filter_ifg=True,
    multilook=[1, 4],
    warp_kernel="bicubic",
    clip_to_shape=True,
)

INFO:eo_tools.S1.process:Creating directory data/res/test-full-processor/S1_InSAR_2023-09-04-174209__2023-09-16-174209/sar
INFO:eo_tools.S1.process:---- Processing subswath IW1 in VV polarization
INFO:eo_tools.S1.core:S1IWSwath Initialization:
INFO:eo_tools.S1.core:- Read metadata file data/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.zip/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.SAFE/annotation/s1a-iw1-slc-vv-20230904t174210-20230904t174235-050181-060a2d-004.xml
INFO:eo_tools.S1.core:- Read calibration file data/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.zip/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.SAFE/annotation/calibration/calibration-s1a-iw1-slc-vv-20230904t174210-20230904t174235-050181-060a2d-004.xml
INFO:eo_tools.S1.core:- Set up raster path zip://data/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_060A2D_E88F.zip/S1A_IW_SLC__1SDV_20230904T174209_20230904T174236_050181_0